# Project 1 FYS-STK4155
### Trial run Kjersti

In [129]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

#### Part a) OLS for the Runge function

* OLS regression analysis using polynomials in x up to order 15 or higher. Add stochastic noise.
* Explore the dependence on the number of data points and the polynomial degree
* Evalute the MSE and R^2 scores. Plot them as functions of polynomial degree.
* Plot the parameters $\theta$ as you increase the order of the polynomial. Comment the results.
* You have to include a scaling/centering of the data.
* Present a critical discussion of why and how you have scaled the data.
* You have to split into test and training data.

In [166]:
n_vals = np.arange(10, 1500, 100)  # number of data points
p_vals = np.arange(2, 16) # polynomial degree 

In [134]:
def make_data(n):
    '''
    Make a dataset with a given number (n) datapoints
    of the Runge function.
    '''
    x = np.linspace(-1, 1, n)
    y = 1/(1 + 25*x**2) + np.random.normal(0, 0.1)

    return x, y

def polynomial_features(x, p):
    '''
    Make a design matrix.
    '''
    n = len(x)
    X = np.zeros((n, p + 1))
    X[:, 0] = 1
    for i in range(1, p+1):
        X[:, i] = x**i

    return X

def OLS_parameters(X, y):
    ''' 
    Find the OLS parameters
    '''
    X_T = np.transpose(X)
    X_T_X = X_T @ X

    return np.linalg.pinv(X_T_X) @ X_T @ y

def MSE(y_data, y_pred):
    ''' 
    Mean square error
    '''
    return np.mean((y_data - y_pred)**2)

def R2(y_data, y_pred):
    return 1 - (np.sum((y_data - y_pred)**2))/(np.sum((y_data - np.mean(y_data))**2))

def standardize(X, y):
    # Standardize features (zero mean, unit variance for each feature)
    X_mean = X.mean(axis=0) # The mean of each column/feature
    X_std = X.std(axis=0)
    X_std[X_std == 0] = 1  # safeguard to avoid division by zero for constant features
    X_norm = (X - X_mean) / X_std

    # Center the target to zero mean (optional, to simplify intercept handling)
    y_mean = y.mean()
    y_centered = y - y_mean

    return X_norm, y_centered

def split_n_train(X, y, size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)

    return X_train, X_test, y_train, y_test


In [174]:
results = []

for n in n_vals:
    x, y = make_data(n)

    for p in p_vals:
        X = polynomial_features(x, p)
        X, y = standardize(X, y)
        X_train, X_test, y_train, y_test = split_n_train(X, y, size=0.2)

        theta = OLS_parameters(X_train, y_train)
        y_pred = X_test @ theta

        results.append({"n": n, "p": p, "theta": theta, "MSE": MSE(y_test, y_pred), "R2": R2(y_test, y_pred)})

df = pd.DataFrame(results)

/var/folders/tp/bn0hk5sx7fq7r171hlv9tzq40000gn/T/ipykernel_86225/942559193.py:39: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - (np.sum((y_data - y_pred)**2))/(np.sum((y_data - np.mean(y_data))**2))
